# Práctica del Padrón

## Creación de tablas en Hive

In [ ]:
//Crea la base de datos
create database datos_padron

//Creación de tablas almacenados en csv o parquet en HDFS
create table padron_txt(
COD_DISTRITO string,
DESC_DISTRITO string,
COD_DIST_BARRIO string,
DESC_BARRIO string,
COD_BARRIO string,
COD_DIST_SECCION string,
COD_SECCION string,
COD_EDAD_INT string,
EspanolesHombres string,
EspanolesMujeres string,
ExtranjerosHombres string,
ExtranjerosMujeres string)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\;'
STORED AS TEXTFILE;


create table padron_txt2(
COD_DISTRITO int,
DESC_DISTRITO string,
COD_DIST_BARRIO int,
DESC_BARRIO string,
COD_BARRIO int,
COD_DIST_SECCION int,
COD_SECCION int,
COD_EDAD_INT int,
EspanolesHombres int,
EspanolesMujeres int,
ExtranjerosHombres int,
ExtranjerosMujeres int)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = '\073',
'quoteChar' = '"',
'escapeChar' = '\\')
STORED AS TEXTFILE
tblproperties("skip.header.line.count"="1");

Load data inpath '/user/cloudera/hive/Rango_Edades_Seccion_202106.csv' into table padron_txt2;


create table padron_txt3
STORED AS TEXTFILE
AS SELECT COD_DISTRITO, TRIM(DESC_DISTRITO) as DESC_DISTRITO , COD_DIST_BARRIO, TRIM(DESC_BARRIO) as DESC_BARRIO, COD_BARRIO, COD_DIST_SECCION, COD_SECCION, COD_EDAD_INT,
CASE WHEN EspanolesHombres = '' THEN 0 ELSE EspanolesHombres END AS EspanolesHombres,
CASE WHEN EspanolesMujeres = '' THEN 0 ELSE EspanolesMujeres END AS EspanolesMujeres,
CASE WHEN ExtranjerosHombres = '' THEN 0 ELSE ExtranjerosHombres END AS ExtranjerosHombres,
CASE WHEN ExtranjerosMujeres = '' THEN 0 ELSE ExtranjerosMujeres END AS ExtranjerosMujeres
FROM padron_txt2; 

CREATE TABLE datos_padron.padron_txt_reg (COD_DISTRITO INT,  DESC_DISTRITO STRING,
COD_DIST_BARRIO INT, DESC_BARRIO STRING,
COD_BARRIO INT, COD_DIST_SECCION INT,
COD_SECCION INT, COD_EDAD_INT INT,
EspanolesHombres INT, EspanolesMujeres INT,
ExtranjerosHombres INT, ExtranjerosMujeres INT)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES ("input.regex"="'(\\d+)'\;'(.*?)\\s*'\;'(\\d+)'\;'(.*?)\s*'\;'(\\d+)'\;'(\\d+)'\;'(\\d+)'\;'(\\d+)'\;'(.*?)'\;'(.*?)'\;'(.*?)'\;'(.*?)", "serialization.encoding"="UTF-8")
STORED AS TEXTFILE;

Load data inpath '/user/cloudera/hive/Rango_Edades_Seccion_202106.csv' into table datos_padron.padron_txt_reg;

CTAS es una forma de crear una tabla en base al contenido de otra mediante una consulta selectiva.



In [ ]:
create table padron_parquet 
STORED AS PARQUET
as SELECT * from padron_txt;

create table padron_parquet2
STORED AS PARQUET
as SELECT * from padron_txt2;

create table padron_parquet3
STORED AS PARQUET
as SELECT * from padron_txt3;

//Para mirar el tamaño de las tablas

hdfs dfs -du -s -h /user/hive/warehouse/datos_padron.db/padron_txt
hdfs dfs -du -s -h /user/hive/warehouse/datos_padron.db/padron_parquet


### Impala

Impala es otro motor SQL que, de igual forma que Hive, opera consultas sobre
HDFS. Hive e Impala son herramientas diferentes, y cada uno ofrece unas
características importantes:

Impala fue creado para resolver los problemas de tiempo de ejecución elevadas de
Hive, ya que opera las consultas de datos directamente en el clúster de HDFS, sin
utilizar los Jobs con MapReduce. Sin embargo, pierde la tolerancia de fallos que
facilitaba MapReduce, por lo que si una query falla por el sistema, es necesario
relanzar la consulta. Como resultado, Impala suele ser hasta 20 veces más rápido
que Hive.

Normalmente, estas dos herramientas se utilizan en proyectos grandes, ya que en
caso de realizar procesos ETL para extraer, transformar y cargar datos, buscando
robustez sin importar el tiempo computacional, es muy conveniente utilizar Hive. Y
luego con Impala para realizar consultas rápidas tras cargar los datos con Hive.


⦁	La diferencia entre Impala y Hive es que Impala ejecuta las queries directamente en un clúster y hive en un MapReduce. Como podemos recordar, en los ejericicos de Hijve, ejecutaban un reducer por cada agrupación (sum, count, etc).

⦁	Por ello Impala es mucho más rápido de Hive o Pig

⦁	Para utilizar Impala no es necesario saber programar. Únicamente es necesario saber realizar queries.

⦁	Impala, como Hive, operan sobre los mismos datos: tablas en HDFS, y metadatos en el Metastore.


La INVALIDATE METADATAdeclaración marca los metadatos de una o todas las tablas como obsoletos. La próxima vez que el servicio Impala realiza una consulta en una tabla cuyos metadatos se invalidan, Impala vuelve a cargar los metadatos asociados antes de que continúe la consulta. Como esta es una operación muy costosa en comparación con la actualización incremental de metadatos realizada por la REFRESHdeclaración, cuando sea posible, prefiera en REFRESH lugar de INVALIDATE METADATA.

INVALIDATE METADATA es necesario cuando se realizan los siguientes cambios fuera de Impala, en Hive y otros clientes de Hive, como SparkSQL:

Cambios en los metadatos de las tablas existentes.

Se agregan nuevas tablas e Impala las usará.

Se cambian los privilegios de centinela SERVERo DATABASEnivel.

Bloquea los cambios de metadatos, pero los archivos siguen siendo los mismos (reequilibrio de HDFS).

Los frascos UDF cambian.

Algunas tablas ya no se consultan y desea eliminar sus metadatos de las cachés del coordinador y del catálogo para reducir los requisitos de memoria.


In [ ]:
//HIVE
SELECT DESC_DISTRITO, DESC_BARRIO, sum(EspanolesHombres) as TotalESPH, sum(EspanolesMujeres) as TotalESPM, sum(ExtranjerosHombres) as TotalEXH, sum(ExtranjerosMujeres) as EXM FROM padron_txt3 GROUP BY DESC_DISTRITO, DESC_BARRIO
SELECT DESC_DISTRITO, DESC_BARRIO, sum(EspanolesHombres) as TotalESPH, sum(EspanolesMujeres) as TotalESPM, sum(ExtranjerosHombres) as TotalEXH, sum(ExtranjerosMujeres) as EXM FROM padron_parquet GROUP BY DESC_DISTRITO, DESC_BARRIO
//IMPALA
SELECT DESC_DISTRITO, DESC_BARRIO, sum(CAST(EspanolesHombres AS INT)) as TotalESPH, sum(CAST(EspanolesMujeres AS INT)) as TotalESPM, sum(CAST(ExtranjerosHombres AS INT)) as TotalEXH, sum(CAST(ExtranjerosMujeres AS INT)) as TotalEXM FROM padron_txt3 GROUP BY DESC_DISTRITO, DESC_BARRIO
SELECT DESC_DISTRITO, DESC_BARRIO, sum(CAST(EspanolesHombres AS INT)) as TotalESPH, sum(CAST(EspanolesMujeres AS INT)) as TotalESPM, sum(CAST(ExtranjerosHombres AS INT)) as TotalEXH, sum(CAST(ExtranjerosMujeres AS INT)) as TotalEXM FROM padron_parquet3 GROUP BY DESC_DISTRITO, DESC_BARRIO

## Tablas particionadas

En este ejemplo particionamos el COD_Distrito en COD_Barrio, por lo que tendremosdentro del warehouse de Hive en HDFS una cantidad de ficheros distribuidos por estos dos campos y hacer la búsuqeda más reducida y eficiente.

In [ ]:

SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=100000;
SET hive.exec.max.dynamic.partitions.pernode=100000;

create table padron_particionado(
DESC_DISTRITO string,
COD_DIST_BARRIO string,
DESC_BARRIO string,
COD_DIST_SECCION string,
COD_SECCION string,
COD_EDAD_INT string,
EspanolesHombres string,
EspanolesMujeres string,
ExtranjerosHombres string,
ExtranjerosMujeres string)
PARTITIONED BY (COD_DISTRITO string, COD_BARRIO string)
STORED AS PARQUET;


INSERT OVERWRITE TABLE padron_particionado PARTITION(COD_DISTRITO, COD_BARRIO)
SELECT DESC_DISTRITO,
COD_DIST_BARRIO,
DESC_BARRIO,
COD_DIST_SECCION,
COD_SECCION,
COD_EDAD_INT,
EspanolesHombres,
EspanolesMujeres,
ExtranjerosHombres,
ExtranjerosMujeres,
COD_DISTRITO,
COD_BARRIO
FROM padron_parquet3;

//Por qué cuando creo tablas en Hive, no se me actualizan en Impala? No se supone que leen los mismos metadatos alojados en HDFS?

In [ ]:
//Parquet3
SELECT COD_DISTRITO, 
COD_BARRIO, sum(EspanolesHombres) as TotalESPH, sum(EspanolesMujeres) as TotalESPM, 
sum(ExtranjerosHombres) as TotalEXH, sum(ExtranjerosMujeres) as EXM 
FROM padron_parquet3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Time taken: 23.303 seconds, Fetched: 35 row(s)

//Particionado
SELECT COD_DISTRITO, 
COD_BARRIO, sum(EspanolesHombres) as TotalESPH, sum(EspanolesMujeres) as TotalESPM, 
sum(ExtranjerosHombres) as TotalEXH, sum(ExtranjerosMujeres) as EXM 
FROM padron_particionado WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Time taken: 23.676 seconds

//TextFile
SELECT COD_DISTRITO, 
COD_BARRIO, sum(EspanolesHombres) as TotalESPH, sum(EspanolesMujeres) as TotalESPM, 
sum(ExtranjerosHombres) as TotalEXH, sum(ExtranjerosMujeres) as EXM 
FROM padron_txt3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Time taken: 23.72 seconds, Fetched: 35 row(s)

//

In [ ]:
//En IMPALA

//Parquet3
SELECT COD_DISTRITO, 
COD_BARRIO, sum(CAST(EspanolesHombres AS INT)) as TotalESPH, sum(CAST(EspanolesMujeres AS INT)) as TotalESPM, 
sum(CAST(ExtranjerosHombres AS INT)) as TotalEXH, sum(CAST(ExtranjerosMujeres AS INT)) as EXM 
FROM padron_parquet3 WHERE COD_DISTRITO IN ('1', '5', '10', '19', '6', '21')
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Fetched 35 row(s) in 0.75s

//Particionado
SELECT COD_DISTRITO, 
COD_BARRIO, sum(CAST(EspanolesHombres AS INT)) as TotalESPH, sum(CAST(EspanolesMujeres AS INT)) as TotalESPM, 
sum(CAST(ExtranjerosHombres AS INT)) as TotalEXH, sum(CAST(ExtranjerosMujeres AS INT)) as EXM 
FROM padron_particionado WHERE COD_DISTRITO IN ('1', '5', '10', '19', '6', '21')
GROUP BY COD_DISTRITO, COD_BARRIO ;
// Fetched 35 row(s) in 0.83s


//TextFile
SELECT COD_DISTRITO, 
COD_BARRIO, sum(CAST(EspanolesHombres AS INT)) as TotalESPH, sum(CAST(EspanolesMujeres AS INT)) as TotalESPM, 
sum(CAST(ExtranjerosHombres AS INT)) as TotalEXH, sum(CAST(ExtranjerosMujeres AS INT)) as EXM 
FROM padron_txt3 WHERE COD_DISTRITO IN ('1', '5', '10', '19', '6', '21')
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Fetched 35 row(s) in 3.66s
//Es más tarde, ya que hay que leer todo el conjunto de datos, y ya luego estrcuturar las columnas.
//En cambio en parquet únicamente lees las columnas específicas del conjunto de datos y la estructura columnar ya está hecho.

In [ ]:
//Prueba de rendimiento

SELECT * from padron_parquet where COD_DISTRITO = 1 AND COD_BARRIO = 1;
SELECT * from padron_particionado where COD_DISTRITO = 1 AND COD_BARRIO = 1;

// Para tabla particionada
SELECT COD_DISTRITO, COD_BARRIO, max(EspanolesHombres) as MAXESPH, min(EspanolesHombres) as MINESPH,
avg(EspanolesHombres) as AVGESPH, count(*) as TOTAL
FROM padron_particionado WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO;
//Time taken: 27.471 seconds

SELECT COD_DISTRITO, COD_BARRIO, max(EspanolesMujeres) as MAXESPM, min(EspanolesMujeres) as MINESPM,
avg(EspanolesMujeres) as AVGESPM, count(*) as TOTAL
FROM padron_particionado WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Time taken: 21.28 seconds

SELECT COD_DISTRITO, COD_BARRIO, max(ExtranjerosHombres) as MAXEXH, min(ExtranjerosHombres) as MINEXH,
avg(ExtranjerosHombres) as AVGEXH, count(*) as TOTAL
FROM padron_particionado WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;

SELECT COD_DISTRITO, COD_BARRIO, max(ExtranjerosMujeres) as MAXEXM, min(ExtranjerosMujeres) as MINEXM,
avg(ExtranjerosMujeres) as AVGEXM, count(*) as TOTAL
FROM padron_particionado WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;

// Para tabla parquet
SELECT COD_DISTRITO, COD_BARRIO, max(EspanolesHombres) as MAXESPH, min(EspanolesHombres) as MINESPH,
avg(EspanolesHombres) as AVGESPH, count(*) as TOTAL
FROM padron_parquet3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO;
//Time taken: 22.975 seconds, Fetched: 35 row(s)

SELECT COD_DISTRITO, COD_BARRIO, max(EspanolesMujeres) as MAXESPM, min(EspanolesMujeres) as MINESPM,
avg(EspanolesMujeres) as AVGESPM, count(*) as TOTAL
FROM padron_parquet3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Time taken: 21.717 seconds, Fetched: 35 row(s)


SELECT COD_DISTRITO, COD_BARRIO, max(ExtranjerosHombres) as MAXEXH, min(ExtranjerosHombres) as MINEXH,
avg(ExtranjerosHombres) as AVGEXH, count(*) as TOTAL
FROM padron_parquet3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;

SELECT COD_DISTRITO, COD_BARRIO, max(ExtranjerosMujeres) as MAXEXM, min(ExtranjerosMujeres) as MINEXM,
avg(ExtranjerosMujeres) as AVGEXM, count(*) as TOTAL
FROM padron_parquet3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;

// Para tabla textfile
SELECT COD_DISTRITO, COD_BARRIO, max(EspanolesHombres) as MAXESPH, min(EspanolesHombres) as MINESPH,
avg(EspanolesHombres) as AVGESPH, count(*) as TOTAL
FROM padron_txt3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO;
//Time taken: 20.831 seconds, Fetched: 35 row(s)

SELECT COD_DISTRITO, COD_BARRIO, max(EspanolesMujeres) as MAXESPM, min(EspanolesMujeres) as MINESPM,
avg(EspanolesMujeres) as AVGESPM, count(*) as TOTAL
FROM padron_txt3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;
//Time taken: 32.027 seconds, Fetched: 35 row(s)

SELECT COD_DISTRITO, COD_BARRIO, max(ExtranjerosHombres) as MAXEXH, min(ExtranjerosHombres) as MINEXH,
avg(ExtranjerosHombres) as AVGEXH, count(*) as TOTAL
FROM padron_txt3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;

SELECT COD_DISTRITO, COD_BARRIO, max(ExtranjerosMujeres) as MAXEXM, min(ExtranjerosMujeres) as MINEXM,
avg(ExtranjerosMujeres) as AVGEXM, count(*) as TOTAL
FROM padron_txt3 WHERE COD_DISTRITO IN (1, 5, 10, 19, 6, 21)
GROUP BY COD_DISTRITO, COD_BARRIO ;


## 5 Tablas Gestionadas y no gestionadas

In [ ]:
create table numeros_tbl(
col1 int,
col2 int,
col3 int)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

Load data inpath '/user/cloudera/test/datos1.txt' into table numeros.numeros_tbl;

create external table numeros.numeros_tbl(
col1 int,
col2 int,
col3 int)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/cloudera/test';

En Hive cuando cargas los datos de un fichero, recoge ese fichero de HDFS y lo almacena en el warehouse de Hive.
Cuando borramos la tabla, se borra el fichero.

Sin embargo, en caso de crearlo externo, al borrarlo el fichero vuelve a estar en el directorio de HDFS que estaba anteriormente.

Cuando realizamos un select de la tabla, sin haber cargado ningún dato, si lo situamos en el mismo directorio que contiene los datos del fichero, se cargan automáticamente


## 6 Spark

In [1]:
val padron_notClean = (spark.read.format("csv")
 .option("header", "true")
 .option("delimiter", ";")
 .option("inferSchema", "true")
 //.enableHiveSupport()
 .load("Rango_Edades_Seccion_202106.csv"))

val padron_df = padron_notClean.select(col("COD_DISTRITO"), 
                                   trim(col("DESC_DISTRITO")).alias("DESC_DISTRITO"), 
                                   col("COD_DIST_BARRIO"),
                                   trim(col("DESC_BARRIO")).alias("DESC_BARRIO"),
                                   col("COD_BARRIO"),
                                   col("COD_DIST_SECCION"),
                                   col("COD_SECCION"),
                                   col("COD_EDAD_INT"),
                                   when(col("EspanolesHombres").isNull, 0).otherwise(col("EspanolesHombres")).alias("EspanolesHombres"),
                                   when(col("EspanolesMujeres").isNull, 0).otherwise(col("EspanolesMujeres")).alias("EspanolesMujeres"),
                                   when(col("ExtranjerosHombres").isNull, 0).otherwise(col("ExtranjerosHombres")).alias("ExtranjerosHombres"),
                                   when(col("ExtranjerosMujeres").isNull, 0).otherwise(col("ExtranjerosMujeres")).alias("ExtranjerosMujeres")
                                  )

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002844.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1624520846037)
SparkSession available as 'spark'


padron_notClean: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]
padron_df: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [38]:
padron_df.printSchema()

root
 |-- COD_DISTRITO: integer (nullable = true)
 |-- DESC_DISTRITO: string (nullable = true)
 |-- COD_DIST_BARRIO: integer (nullable = true)
 |-- DESC_BARRIO: string (nullable = true)
 |-- COD_BARRIO: integer (nullable = true)
 |-- COD_DIST_SECCION: integer (nullable = true)
 |-- COD_SECCION: integer (nullable = true)
 |-- COD_EDAD_INT: integer (nullable = true)
 |-- EspanolesHombres: integer (nullable = true)
 |-- EspanolesMujeres: integer (nullable = true)
 |-- ExtranjerosHombres: integer (nullable = true)
 |-- ExtranjerosMujeres: integer (nullable = true)



In [102]:
spark.sql("""
SELECT DISTINCT DESC_BARRIO, rank 
 FROM ( 
 SELECT DISTINCT DESC_BARRIO, row_number() 
 OVER (PARTITION BY DESC_BARRIO ORDER BY DESC_BARRIO) as rank 
 FROM padron
 ) t 
""").show()


+------------+----+
| DESC_BARRIO|rank|
+------------+----+
|VALDEFUENTES|   1|
|VALDEFUENTES|   2|
|VALDEFUENTES|   3|
|VALDEFUENTES|   4|
|VALDEFUENTES|   5|
|VALDEFUENTES|   6|
|VALDEFUENTES|   7|
|VALDEFUENTES|   8|
|VALDEFUENTES|   9|
|VALDEFUENTES|  10|
|VALDEFUENTES|  11|
|VALDEFUENTES|  12|
|VALDEFUENTES|  13|
|VALDEFUENTES|  14|
|VALDEFUENTES|  15|
|VALDEFUENTES|  16|
|VALDEFUENTES|  17|
|VALDEFUENTES|  18|
|VALDEFUENTES|  19|
|VALDEFUENTES|  20|
+------------+----+
only showing top 20 rows



In [22]:
//Crear vista temporal del padron
padron_df.createOrReplaceTempView("padron")

In [25]:
//Cuenta todos los diferentes barrios
spark.sql("SELECT count(DISTINCT DESC_BARRIO) as Total_Barrios FROM padron").show()

+-------------+
|Total_Barrios|
+-------------+
|          132|
+-------------+



In [31]:
//Añade una columna nueva longitud que cuente la cantidad de caracteres
val padron_longitud = padron_df.withColumn("longitud", expr("char_length(DESC_DISTRITO)"))
padron_longitud.createOrReplaceTempView("padron_longitud")

padron_longitud: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]


In [33]:
spark.sql("SELECT DISTINCT DESC_DISTRITO, longitud FROM padron_longitud").show()

+-------------------+--------+
|      DESC_DISTRITO|longitud|
+-------------------+--------+
|         ARGANZUELA|      10|
|FUENCARRAL-EL PARDO|      19|
|              USERA|       5|
|          SALAMANCA|       9|
| PUENTE DE VALLECAS|      18|
|  VILLA DE VALLECAS|      17|
|           CHAMBERI|       8|
|          VICALVARO|       9|
|             RETIRO|       6|
|             CENTRO|       6|
|SAN BLAS-CANILLEJAS|      19|
|          CHAMARTIN|       9|
|             LATINA|       6|
|          MORATALAZ|       9|
|            BARAJAS|       7|
|             TETUAN|       6|
|      CIUDAD LINEAL|      13|
|          HORTALEZA|       9|
|         VILLAVERDE|      10|
|        CARABANCHEL|      11|
+-------------------+--------+
only showing top 20 rows



In [35]:
//Crear una columna que ponga un 5 para cada fila de la tabla
val padron_5 = padron_longitud.withColumn("valor_5", lit(5))
padron_5.createOrReplaceTempView("padron_5")
padron_5.drop("valor_5")

padron_5: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 12 more fields]


In [38]:
spark.sql("select DISTINCT DESC_DISTRITO, valor_5 FROM padron_5").show()

+-------------------+-------+
|      DESC_DISTRITO|valor_5|
+-------------------+-------+
|              USERA|      5|
|             TETUAN|      5|
|      CIUDAD LINEAL|      5|
|          VICALVARO|      5|
|SAN BLAS-CANILLEJAS|      5|
| PUENTE DE VALLECAS|      5|
|             CENTRO|      5|
|          SALAMANCA|      5|
|        CARABANCHEL|      5|
|          HORTALEZA|      5|
|         ARGANZUELA|      5|
|             RETIRO|      5|
|          MORATALAZ|      5|
|    MONCLOA-ARAVACA|      5|
|FUENCARRAL-EL PARDO|      5|
|          CHAMARTIN|      5|
|            BARAJAS|      5|
|             LATINA|      5|
|         VILLAVERDE|      5|
|           CHAMBERI|      5|
+-------------------+-------+
only showing top 20 rows



In [40]:
val padron_partitioned = padron_df.repartition($"COD_DISTRITO", $"COD_BARRIO")
padron_partitioned.cache()

padron_partitioned: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]
res32: padron_partitioned.type = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [44]:
//6.10
padron_partitioned.select($"COD_DISTRITO", $"COD_BARRIO", $"EspanolesHombres", $"EspanolesMujeres", $"ExtranjerosHombres", $"ExtranjerosMujeres")
 .groupBy($"COD_DISTRITO", $"COD_BARRIO")
 .agg(sum($"EspanolesHombres").alias("TotalESPH"), sum($"EspanolesMujeres").alias("TotalESPM"), sum($"ExtranjerosHombres").alias("TotalEXH"), sum($"ExtranjerosMujeres").alias("TotalEXM"))
 .orderBy($"TotalEXM".desc, $"TotalEXH".desc)
 .show(false)

+------------+----------+---------+---------+--------+--------+
|COD_DISTRITO|COD_BARRIO|TotalESPH|TotalESPM|TotalEXH|TotalEXM|
+------------+----------+---------+---------+--------+--------+
|13          |2         |13875    |15544    |6732    |6961    |
|10          |4         |25000    |29776    |5372    |6257    |
|15          |2         |23291    |27325    |5477    |6198    |
|11          |4         |15988    |19466    |5356    |6000    |
|1           |2         |16693    |16666    |8019    |5718    |
|13          |6         |17200    |19472    |5332    |5603    |
|17          |1         |17087    |19060    |5031    |5236    |
|15          |1         |17715    |21818    |3924    |4783    |
|10          |2         |15169    |17728    |4097    |4653    |
|11          |5         |12525    |14240    |4230    |4624    |
|11          |3         |14487    |16574    |4019    |4395    |
|8           |6         |26955    |29190    |3532    |4282    |
|1           |5         |12542    |12631

In [45]:
//Eliminar de cache
padron_partitioned.unpersist()

res37: padron_partitioned.type = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [60]:
val padron_totalEspH = padron_df.select($"DESC_DISTRITO".alias("dis"), $"DESC_BARRIO".alias("bar"), $"EspanolesHombres")
 .groupBy($"dis", $"bar")
 .agg(sum($"EspanolesHombres").alias("Total"))

val padron_join = padron_df.join(padron_totalEspH.as("EspH"), $"EspH.dis" === $"DESC_DISTRITO" && $"EspH.bar" === $"DESC_BARRIO")

padron_totalEspH: org.apache.spark.sql.DataFrame = [dis: string, bar: string ... 1 more field]
padron_join: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 13 more fields]


In [129]:
padron_join.select($"DESC_DISTRITO", $"DESC_BARRIO", $"Total").distinct.show()

+-------------------+-----------------+-----+
|      DESC_DISTRITO|      DESC_BARRIO|Total|
+-------------------+-----------------+-----+
|FUENCARRAL-EL PARDO|       MIRASIERRA|16011|
|          SALAMANCA|       CASTELLANA| 6140|
|SAN BLAS-CANILLEJAS|      EL SALVADOR| 4870|
|          HORTALEZA|     VALDEFUENTES|28457|
|    MONCLOA-ARAVACA|    CASA DE CAMPO| 5421|
|          MORATALAZ|       MARROQUINA|11426|
|             TETUAN|    BELLAS VISTAS|10202|
|          VICALVARO|     EL CA?AVERAL| 2819|
|             CENTRO|         JUSTICIA| 7097|
|             CENTRO|      UNIVERSIDAD|12542|
|      CIUDAD LINEAL|          ATALAYA|  600|
|             TETUAN|       BERRUGUETE| 8677|
|          VICALVARO|CASCO H.VICALVARO|13391|
|          CHAMARTIN|         CASTILLA| 7187|
|         ARGANZUELA|          LEGAZPI| 8861|
|SAN BLAS-CANILLEJAS|           HELLIN| 3609|
|          CHAMARTIN|   HISPANOAMERICA|13254|
|  VILLA DE VALLECAS|    SANTA EUGENIA|10375|
|             CENTRO|      EMBAJAD

In [123]:
import org.apache.spark.sql.expressions.Window
val padron_totalEspH = padron_df
 .withColumn("Total", sum($"EspanolesHombres").over(Window.partitionBy($"DESC_DISTRITO", $"DESC_BARRIO")))

import org.apache.spark.sql.expressions.Window
padron_totalEspH: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]


In [127]:
padron_totalEspH.select("DESC_DISTRITO", "DESC_BARRIO", "Total").distinct.show()

+-------------------+-----------------+-----+
|      DESC_DISTRITO|      DESC_BARRIO|Total|
+-------------------+-----------------+-----+
|FUENCARRAL-EL PARDO|       MIRASIERRA|16011|
|          SALAMANCA|       CASTELLANA| 6140|
|SAN BLAS-CANILLEJAS|      EL SALVADOR| 4870|
|          HORTALEZA|     VALDEFUENTES|28457|
|    MONCLOA-ARAVACA|    CASA DE CAMPO| 5421|
|          MORATALAZ|       MARROQUINA|11426|
|             TETUAN|    BELLAS VISTAS|10202|
|          VICALVARO|     EL CA?AVERAL| 2819|
|             CENTRO|         JUSTICIA| 7097|
|             CENTRO|      UNIVERSIDAD|12542|
|      CIUDAD LINEAL|          ATALAYA|  600|
|             TETUAN|       BERRUGUETE| 8677|
|          VICALVARO|CASCO H.VICALVARO|13391|
|          CHAMARTIN|         CASTILLA| 7187|
|         ARGANZUELA|          LEGAZPI| 8861|
|SAN BLAS-CANILLEJAS|           HELLIN| 3609|
|          CHAMARTIN|   HISPANOAMERICA|13254|
|  VILLA DE VALLECAS|    SANTA EUGENIA|10375|
|             CENTRO|      EMBAJAD

In [5]:
padron_df.printSchema()

root
 |-- COD_DISTRITO: integer (nullable = true)
 |-- DESC_DISTRITO: string (nullable = true)
 |-- COD_DIST_BARRIO: integer (nullable = true)
 |-- DESC_BARRIO: string (nullable = true)
 |-- COD_BARRIO: integer (nullable = true)
 |-- COD_DIST_SECCION: integer (nullable = true)
 |-- COD_SECCION: integer (nullable = true)
 |-- COD_EDAD_INT: integer (nullable = true)
 |-- EspanolesHombres: integer (nullable = true)
 |-- EspanolesMujeres: integer (nullable = true)
 |-- ExtranjerosHombres: integer (nullable = true)
 |-- ExtranjerosMujeres: integer (nullable = true)



In [13]:
val distritos = Seq("BARAJAS", "CENTRO", "RETIRO")
val padron_pivot = padron_df.groupBy($"COD_EDAD_INT")
 .pivot($"DESC_DISTRITO", distritos)
 .agg(sum($"EspanolesMujeres"))
 .orderBy($"COD_EDAD_INT")

padron_pivot.show()

+------------+-------+------+------+
|COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|
+------------+-------+------+------+
|           0|    157|   245|   283|
|           1|    196|   243|   368|
|           2|    179|   241|   382|
|           3|    231|   232|   399|
|           4|    235|   246|   416|
|           5|    275|   232|   390|
|           6|    236|   262|   474|
|           7|    256|   227|   420|
|           8|    244|   257|   421|
|           9|    276|   254|   457|
|          10|    254|   255|   409|
|          11|    256|   253|   403|
|          12|    278|   283|   448|
|          13|    269|   247|   420|
|          14|    265|   249|   424|
|          15|    284|   279|   408|
|          16|    256|   241|   416|
|          17|    247|   274|   451|
|          18|    249|   288|   385|
|          19|    208|   254|   450|
+------------+-------+------+------+
only showing top 20 rows



distritos: Seq[String] = List(BARAJAS, CENTRO, RETIRO)
padron_pivot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_EDAD_INT: int, BARAJAS: bigint ... 2 more fields]


In [34]:
import org.apache.spark.sql.expressions.Window

padron_pivot
 .withColumn("PERBARAJAS", $"BARAJAS"/(sum($"BARAJAS").over(Window.partitionBy())))
 .withColumn("PERCENTRO", $"BARAJAS"/(sum($"CENTRO").over(Window.partitionBy())))
 .withColumn("PERRETIRO", $"BARAJAS"/(sum($"RETIRO").over(Window.partitionBy())))
 .show()

+------------+-------+------+------+--------------------+--------------------+--------------------+
|COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|          PERBARAJAS|           PERCENTRO|           PERRETIRO|
+------------+-------+------+------+--------------------+--------------------+--------------------+
|           0|    157|   245|   283|0.006771035494026825|0.003004209720627...|0.002677033778368885|
|           1|    196|   243|   368|0.008453012463880622|0.003750478377344049|0.003342029430320...|
|           2|    179|   241|   382|0.007719843015482...|0.003425181783390739|0.003052159530751...|
|           3|    231|   232|   399|0.009962478975287877|0.004420206659012...|0.003938820400020461|
|           4|    235|   246|   416| 0.01013498943373442|0.004496747034060467|0.004007025082271898|
|           5|    275|   232|   390|0.011860094018199853|0.005262150784538845|0.004689071904786264|
|           6|    236|   262|   474|0.010178117048346057|0.004515882127822426|0.004024076252834757|


import org.apache.spark.sql.expressions.Window


In [37]:
padron_df.repartition($"COD_DISTRITO", $"COD_BARRIO").write.format("csv").option("encoding", "UTF-8").save("Csv/padron.csv")

In [38]:
padron_df.repartition($"COD_DISTRITO", $"COD_BARRIO").write.format("parquet").option("encoding", "UTF-8").save("Parquet/padron.parquet")

In [39]:
spark.sql("create database datos_padron")

res28: org.apache.spark.sql.DataFrame = []


In [42]:
spark.sql("use datos_padron")

res31: org.apache.spark.sql.DataFrame = []


In [55]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SparkSession

import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SparkSession


In [60]:
val conf = new SparkConf()
      .set("spark.sql.catalogImplementation","hive")
      .setMaster("local[*]")
      .setAppName("Hive Example")

val spark2 = SparkSession.builder()
      .config(conf)
      .enableHiveSupport()
      .getOrCreate()

conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@6c37f14a
spark2: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2bf1345d


In [61]:
spark2.sql("""create table padron_txt(
COD_DISTRITO string,
DESC_DISTRITO string,
COD_DIST_BARRIO string,
DESC_BARRIO string,
COD_BARRIO string,
COD_DIST_SECCION string,
COD_SECCION string,
COD_EDAD_INT string,
EspanolesHombres string,
EspanolesMujeres string,
ExtranjerosHombres string,
ExtranjerosMujeres string)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\;'
STORED AS TEXTFILE;""")

org.apache.spark.sql.AnalysisException:  Hive support is required to CREATE Hive TABLE (AS SELECT);